In [1]:
# 필요하시면 인스톨
!pip install selenium
!pip install webdriver-manager

In [8]:
# 라이브러리 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager


from tqdm.notebook import tqdm
from tqdm import tqdm
import time
import pandas as pd 

In [9]:
# 엑셀 파일 읽기
movie_df = pd.read_excel('Movie_DateBase.xlsx')

# 인덱스 범위 지정
index_ranges = [(3500, 4585+1)]  # 예시: 인덱스 범위를 원하는대로 설정
start_idx, end_idx = index_ranges[0]  # 사용하고자 하는 인덱스 범위

# 웹 드라이버 실행
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# 와챠피디아 URL로 이동
url = 'https://pedia.watcha.com/ko-KR'
driver.get(url)
time.sleep(1)
# 
# 해당 범위의 영화 데이터프레임 선택
subset_movie_df = movie_df[3500:4585+1]

# 해당 범위의 영화 별점 검색 및 저장
watch_aver = []  # 영화 별점을 저장할 리스트 초기화

try:
    for movie_name in tqdm(subset_movie_df['Name'], desc="Processing Movies"):
        print(f"검색할 영화 이름: {movie_name}")  # 디버깅을 위한 로그

        # 와챠피디아 검색창 찾기
        search_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="desktop-search-field"]'))
        )
        search_field.clear()  # 이전 입력값 지우기
        search_field.send_keys(movie_name)  # 검색창에 영화 이름 입력
        search_field.send_keys(Keys.ENTER)  # Enter 키로 검색 실행
        time.sleep(2)  # 다음 작업을 위해 2초 대기

        found_movie = False
        a = len(driver.find_elements(By.XPATH, '//*[@id="root"]/div/div[1]/section/section/div[3]/div[1]/section/section[2]/div/div[1]/div/ul/li/a'))

        if a >= 10:
            a = 10

        for i in range(1, a + 1):
            try:
                print(f"검색 결과 {i}번째 항목 확인")  # 디버깅을 위한 로그

                # 영화 링크가 존재하는지 확인 
                movie_link = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH, f'//*[@id="root"]/div/div[1]/section/section/div[3]/div[1]/section/section[1]/div/div[1]/div/ul/li[{i}]/a'))
                )
                movie_link.click()
                time.sleep(1)  # 화면 전환되어 다음작업을 수행할 여유시간을 1초간 부여

                # 와챠피디아의 해당 영화 제목을 변수로 지정
                movie_title_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[1]/section/div/div[2]/div/div[1]/div/div[1]/div[4]/div/h1'))
                )
                movie_title = movie_title_element.text.replace(" ", "").split("(")[0]

                # 와챠피디아의 평균별점 text를 변수로 저장 
                aver_text = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[1]/section/div/div[2]/div/div[1]/div/div[2]/section[1]/div[2]/section[1]/div[2]/div/div[2]'))
                ).text

                # 영화제목이 엑셀 데이터의 영화이름과 일치할 경우
                if movie_title == movie_name.replace(" ", ""):
                    movie_aver_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[1]/section/div/div[2]/div/div[1]/div/div[2]/section[1]/div[2]/section[1]/div[2]/div/div[2]'))
                    )  # 와챠피디아 평균별점 xpath를 변수로 저장
                    movie_aver = movie_aver_element.text  # 평균점수를 변수로 저장

                    print(f"영화 '{movie_name}'의 평점: {movie_aver}")  # 확인용 
                    watch_aver.append(movie_aver)  # watch_aver 변수 리스트에 별점 저장.
                    found_movie = True  # 작업 완료 처리
                    break  # 포문 수행 break

                else:
                    driver.back()
                    time.sleep(2)  # 화면 전환되어 다음작업을 수행할 수 있도록 여유시간을 2초간 부여

            except Exception as inner_e:
                print(f"내부 에러 발생: {inner_e}")
                continue

        if not found_movie:
            watch_aver.append("없음")  # watch_aver에 '없음'으로 저장

            back_to_home = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="root"]/div/div[1]/header/nav/div/div/ul/li'))
            )
            back_to_home.click()  # 메인화면으로 이동              
            time.sleep(2)  # 화면 전환되어 다음작업을 수행할 수 있도록 여유시간을 2초간 부여 

except Exception as e:
    # 예외가 발생할 경우 오류 메시지 출력
    print(f"에러 발생: {e}")  # 예상 외의 문제 발생시, 식별할 수 있도록 '에러발생' 메세지 출력
    watch_aver.append('없음')  # watch_score 변수 리스트에 '없음'으로 저장
    back_to_home = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="root"]/div/div[1]/header/nav/div/div/ul/li' ))  # 와챠피디아 메인화면으로 갈수 있는 xpath 주소
    )
    back_to_home.click()  # 메인화면으로 이동
    time.sleep(2)  # 화면 전환되어 다음작업을 수행할 수 있도록 여유시간을 2초간 부여
finally:
    driver.quit()  # 드라이버 종료

# 결과 저장
subset_movie_df.loc[:, 'Watcha_Average'] = watch_aver  # watch_aver를 subset_movie_df에 추가
subset_movie_df.to_excel(f"movie_aver_{start_idx}_{end_idx}.xlsx", index=False)  # 결과를 엑셀 파일로 저장

Processing Movies:   0%|                                                                      | 0/1086 [00:00<?, ?it/s]

검색할 영화 이름: 방자전
검색 결과 1번째 항목 확인


Processing Movies:   0%|                                                            | 1/1086 [00:03<1:01:54,  3.42s/it]

영화 '방자전'의 평점: 평균 별점
(42.7만명)
검색할 영화 이름: 구르믈 버서난 달처럼
검색 결과 1번째 항목 확인


Processing Movies:   0%|                                                            | 2/1086 [00:06<1:00:55,  3.37s/it]

...

Processing Movies: 100%|████████████████████████████████████████████████████████▉| 1085/1086 [2:15:34<00:03,  3.75s/it]

영화 '웨스트 사이드 스토리'의 평점: 평균 별점
(1.1만명)
검색할 영화 이름: 장미의 곡
검색 결과 1번째 항목 확인


Processing Movies: 100%|█████████████████████████████████████████████████████████| 1086/1086 [2:15:37<00:00,  7.49s/it]

영화 '장미의 곡'의 평점: 평균 별점
(1명)



C:\Users\User\AppData\Local\Temp\ipykernel_23312\1884168374.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_movie_df.loc[:, 'Watcha_Average'] = watch_aver  # watch_aver를 subset_movie_df에 추가


In [4]:
# # 진석님 
# # 엑셀 파일 읽기
# movie_df = pd.read_excel('Movie_DateBase.xlsx') 

# # 인덱스 범위 지정
# index_ranges = [(0, 10)]  # 예시: 인덱스 범위를 원하는대로 설정

# # 웹 드라이버 실행
# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# # 와챠피디아 URL로 이동
# url = 'https://pedia.watcha.com/ko-KR'
# driver.get(url)
# time.sleep(1)



# subset_movie_df = movie_df[0:10]  # 해당 범위의 영화 데이터프레임 선택

# # 해당 범위의 영화 별점 검색 및 저장
# watch_aver = []  # 영화 별점을 저장할 리스트 초기화

# try:
#     for movie_name in tqdm(subset_movie_df['Name'], desc="Processing Movies"):
#         print(f"검색할 영화 이름: {movie_name}")  # 디버깅을 위한 로그

#         # 와챠피디아 검색창 찾기
#         search_field = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.XPATH, '//*[@id="desktop-search-field"]'))
#         )
#         search_field.clear()  # 이전 입력값 지우기
#         search_field.send_keys(movie_name)  # 검색창에 영화 이름 입력
#         search_field.send_keys(Keys.ENTER)  # Enter 키로 검색 실행
#         time.sleep(2)  # 다음 작업을 위해 2초 대기

#         found_movie = False
#         a = len(driver.find_elements(By.XPATH, '//*[@id="root"]/div/div[1]/section/section/div[3]/div[1]/section/section[2]/div/div[1]/div/ul/li/a'))
        
#         if a >= 10 :
#             a = 10
            
#         for i in range(1, a+1):
#             try:
#                 print(f"검색 결과 {i}번째 항목 확인")  # 디버깅을 위한 로그

#                 # 영화 링크가 존재하는지 확인 
#                 movie_link = WebDriverWait(driver, 20).until(
#                     EC.element_to_be_clickable((By.XPATH, f'//*[@id="root"]/div/div[1]/section/section/div[3]/div[1]/section/section[1]/div/div[1]/div/ul/li[{i}]/a'))
#                 )
#                 movie_link.click()
#                 time.sleep(1)  # 화면 전환되어 다음작업을 수행할 여유시간을 1초간 부여

#                 # 와챠피디아의 해당 영화 제목을 변수로 지정
#                 movie_title_element = WebDriverWait(driver, 10).until(
#                     EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[1]/section/div/div[2]/div/div[1]/div/div[1]/div[4]/div/h1'))
#                 )
#                 movie_title = movie_title_element.text.replace(" ", "").split("(")[0]  
#                 # 영화제목의 띄어쓰기를 없애고, '('가 있다면 그 앞쪽text만 변수로 다시 지정
#                 #time.sleep(1)

#                 # 와챠피디아의 평균별점 text를 변수로 저장 
#                 aver_text = WebDriverWait(driver, 10).until(
#                     EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[1]/section/div/div[2]/div/div[1]/div/div[2]/section[1]/div[2]/section[1]/div[2]/div/div[2]'))
#                 ).text
#                 #time.sleep(2)

#                 # 영화제목이 엑셀 데이터의 영화이름과 일치할 경우
#                 if movie_title == movie_name.replace(" ", ""):
#                     movie_aver_element = WebDriverWait(driver, 10).until(
#                         EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[1]/section/div/div[2]/div/div[1]/div/div[2]/section[1]/div[2]/section[1]/div[2]/div/div[2]'))
#                     )  # 와챠피디아 평균별점 xpath를 변수로 저장
#                     movie_aver = movie_aver_element.text  # 평균점수를 변수로 저장
#                     #time.sleep(1)

#                     print(f"영화 '{movie_name}'의 평점: {movie_aver}")  # 확인용 
#                     watch_aver.append(movie_aver)  # watch_aver 변수 리스트에 별점 저장.
#                     found_movie = True  # 작업 완료 처리
#                     break  # 포문 수행 break

#                 else:
#                     driver.back()
#                     time.sleep(2)  # 화면 전환되어 다음작업을 수행할 수 있도록 여유시간을 2초간 부여

#             except Exception as inner_e:
#                 print(f"내부 에러 발생")
#                 continue

#         if not found_movie:
#             watch_aver.append("없음")  # watch_aver에 '없음'으로 저장

#             back_to_home = WebDriverWait(driver, 10).until(
#                 EC.element_to_be_clickable((By.XPATH, '//*[@id="root"]/div/div[1]/header/nav/div/div/ul/li'))
#             )
#             back_to_home.click()  # 메인화면으로 이동              
#             time.sleep(2)  # 화면 전환되어 다음작업을 수행할 수 있도록 여유시간을 2초간 부여 

# except Exception as e:
#     # 예외가 발생할 경우 오류 메시지 출력
#     print(f"에러 발생: {e}")  # 예상 외의 문제 발생시, 식별할 수 있도록 '에러발생' 메세지 출력
#     watch_aver.append('없음')  # watch_score 변수 리스트에 '없음'으로 저장
#     back_to_home = WebDriverWait(driver, 10).until(
#         EC.element_to_be_clickable((By.XPATH, '//*[@id="root"]/div/div[1]/header/nav/div/div/ul/li' ))  # 와챠피디아 메인화면으로 갈수 있는 xpath 주소
#     )
#     back_to_home.click()  # 메인화면으로 이동
#     time.sleep(2)  # 화면 전환되어 다음작업을 수행할 수 있도록 여유시간을 2초간 부여
# finally:
#     driver.quit()  # 드라이버 종료

# # 결과 저장
# subset_movie_df['Watcha_Average'] = watch_aver  # watch_aver를 subset_movie_df에 추가
# subset_movie_df.to_excel(f"movie_aver_{start_idx}_{end_idx}.xlsx", index=False)  # 결과를 엑셀 파일로 저장

Processing Movies:   0%|                                                                        | 0/10 [00:00<?, ?it/s]

검색할 영화 이름: 더 문
검색 결과 1번째 항목 확인


Processing Movies:  10%|██████▍                                                         | 1/10 [00:03<00:31,  3.46s/it]

영화 '더 문'의 평점: 평균 별점
(9.9만명)
검색할 영화 이름: 타임코드 57
검색 결과 1번째 항목 확인


Processing Movies:  20%|████████████▊                                                   | 2/10 [00:06<00:27,  3.38s/it]

...

영화 '신 울트라맨'의 평점: 평균 별점
(477명)
검색할 영화 이름: 마인드 유니버스
검색 결과 1번째 항목 확인


Processing Movies: 100%|███████████████████████████████████████████████████████████████| 10/10 [00:47<00:00,  4.75s/it]

영화 '마인드 유니버스'의 평점: 평균 별점
(92명)



C:\Users\User\AppData\Local\Temp\ipykernel_23312\1256011870.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_movie_df['Watcha_Average'] = watch_aver  # watch_aver를 subset_movie_df에 추가


NameError: name 'start_idx' is not defined